In [48]:
import sys
import os
import argparse
from IPython.display import HTML
import subprocess
import pandas as pd 

In [6]:
# Get the experiment path from command line
CONFIG_FILE = '.config_ipynb'
if os.path.isfile(CONFIG_FILE):
    with open(CONFIG_FILE) as f:
        sys.argv = f.read().split()
else:
    sys.argv = ['test_args.py', 'input_file', '--exp_path', 'test']

parser = argparse.ArgumentParser()
parser.add_argument("input_file",help="Input image, directory, or npy.")
parser.add_argument("--exp_path", type=str, default="test",
                    help="Path to your experiment's runscript")
args = parser.parse_args()
exp_path = args.exp_path
# Remove later, just for testing:
exp_runscript = "/work/ollie/pgierz/awicm_pism_tests/new_hope/scripts/awicm-pism_minitest.run"

# Monitoring for

In [29]:
print(exp_runscript)
exp_top = exp_runscript.split("/")[:-2]
print("/".join(exp_top))

/work/ollie/pgierz/awicm_pism_tests/new_hope/scripts/awicm-pism_minitest.run
/work/ollie/pgierz/awicm_pism_tests/new_hope


In [96]:
os.environ

environ({'MODULE_VERSION_STACK': '3.2.10', 'MANPATH': '/global/AWIsoft/proj/4.9.3/share/man:/global/AWIsoft/intel/2018/intelpython3/share/man:/opt/munge/default/share/man::/global/opt/slurm/default/share/man:', 'XDG_SESSION_ID': '1346098', 'HOSTNAME': 'ollie1', 'TERM': 'xterm-color', 'SHELL': '/bin/bash', 'HISTSIZE': '50000', 'SSH_CLIENT': '172.26.1.151 59168 22', 'QTDIR': '/usr/lib64/qt-3.3', 'QTINC': '/usr/lib64/qt-3.3/include', 'SSH_TTY': '/dev/pts/27', 'QT_GRAPHICSSYSTEM_CHECKED': '1', 'USER': 'pgierz', 'LD_LIBRARY_PATH': '/global/AWIsoft/proj/4.9.3/lib:/opt/cray/lib64:/usr/lib64:/opt/cray/lib64:/usr/lib64:/global/AWIsoft/intel/2018/intelpython3/lib', 'LS_COLORS': 'no=00:rs=0:fi=00:di=01;34:ln=36:mh=04;36:pi=04;01;36:so=04;33:do=04;01;36:bd=01;33:cd=33:or=31:mi=01;37;41:ex=01;36:su=01;04;37:sg=01;04;37:ca=01;37:tw=01;37;44:ow=01;04;34:st=04;37;44:*.7z=01;32:*.ace=01;32:*.alz=01;32:*.arc=01;32:*.arj=01;32:*.bz=01;32:*.bz2=01;32:*.cab=01;32:*.cpio=01;32:*.deb=01;32:*.dz=01;32:*.ear=0

In [11]:
# Read in the definitions from the runscript:
with open(exp_runscript) as f:
    runscript = f.readlines()
machine = [l for l in runscript if "machine_name" in l][0].split("=")[1].split(" ")[0]
machine

'ollie'

In [97]:
# Figure out what batch system we have, and print the queue if we can
function_path = [l for l in runscript if "FUNCTION_PATH" in l][0].split("=")[1].split(" ")[0].strip()
if "slurm" in open(function_path+"/"+machine+".functions").read():
    batch_system = "slurm"
    queue_status = subprocess.getoutput(["squeue -o \"%.18i %.9P %.20j %.8u %.8T  %S %.10M %.9l %.6D %R\" -u "+os.environ["USER"]])
    queue_status = [" ".join(l.split()) for l in queue_status.split("\n")]
    queue_status = [l.split() for l in queue_status]
    queue_df = pd.DataFrame(queue_status[1:])
    queue_df.columns = queue_status[0]
    queue_df["START_TIME"] = pd.to_datetime(queue_df["START_TIME"])
else:
    batch_system = "undefined"
queue_df

,JOBID,PARTITION,NAME,USER,STATE,START_TIME,TIME,TIME_LIMI,NODES,NODELIST(REASON)
0,2473677,mpp,ugh,pgierz,RUNNING,2018-10-10 14:51:06,16:09,3:00:00,20,"prod-[0004-0005,0010-0011,0014-0015,0022-0025,..."
1,2473669,mpp,newer_hope,pgierz,RUNNING,2018-10-10 14:04:10,1:03:05,3:00:00,20,"prod-[0041-0042,0120-0121,0124-0125,0143-0144,..."


In [22]:
    def get_log_output(exp_path):
        expid = exp_path.split("/")[-1]
        stdin, stdout, stderr = client.exec_command('cat '+exp_path+"/scripts/"+expid+".log")
        return stdout

    def generate_dataframe_from_mpiesm_logfile(log):
        log_dataframe = pd.read_table(log,
                                      sep=r" :  | -" ,
                                      skiprows=1,
                                      infer_datetime_format=True,
                                      names=["Date", "Message", "State"],
                                      engine='python', index_col=0)
    middle_column = log_dataframe["Message"].apply(lambda x: pd.Series(str(x).split()))
    log_dataframe.drop("Message", axis=1, inplace=True)
    middle_column.columns = ["Run Number", "Exp Date", "Job ID"]
    log_dataframe = pd.concat([log_dataframe, middle_column], axis=1)
    log_dataframe.set_index(pd.to_datetime(log_dataframe.index), inplace=True)
    return log_dataframe

def compute_effective_throughput(log_dataframe, verbose=False):
    starts = log_dataframe[log_dataframe.State == " start"]
    ends = log_dataframe[log_dataframe.State == " done"]
    starts = starts.index.tolist()
    ends = ends.index.tolist()
    starts = starts[:len(ends)]  # FIXME: the needs to be checked in both directions, and always norm to the shorter list
    diffs = [ends[i] - starts[i] for i in range(len(ends))]
    average_timedelta = sum(diffs, datetime.timedelta(0)) / len(diffs)
    DAY = datetime.timedelta(1)
    throughput = DAY / average_timedelta
    if verbose:
        print("Your run is taking %s on average" % average_timedelta)
        print("this is an effective throughput of %s simulated runs per day, assuming no queue time" % throughput)
    return average_timedelta, throughput

client = paramiko.SSHClient()
client.load_system_host_keys()
client.connect('stan1.awi.de')
stdin, stdout, stderr = client.exec_command('qstat -l')

queue_status = stdout.readlines()
queue_status = [l.split() for l in queue_status]
del queue_status[1]
queue_df = pd.DataFrame(queue_status[1:])
queue_df.columns = queue_status[0]
queue_df
